##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Treinamento multiworker com Estimador

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_estimator"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/distribute/multi_worker_with_estimator.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

> Aviso: os Estimadores não são recomendados para novos códigos.  Os Estimadores executam código `v1.Session`, que é mais difícil de escrever corretamente e pode se comportar se forma inesperada, ainda mais quando usado em conjunto com código do TF 2. Os Estimadores são abarcados pelas [garantias de compatibilidade](https://tensorflow.org/guide/versions), mas não recebem mais correções, exceto para vulnerabilidades de segurança. Confira mais detalhes no [guia de migração](https://tensorflow.org/guide/migrate).

## Visão geral

Observação: embora você possa usar Estimadores com a API do `tf.distribute`, recomenda-se usar o Keras com `tf.distribute`. Confira [Treinamento multiworker com o Keras](multi_worker_with_keras.ipynb). O uso de Estimadores para treinamento com `tf.distribute.Strategy` tem suporte limitado.

Este tutorial demonstra como o `tf.distribute.Strategy` pode ser usado para treinamento multiworker distribuído com o `tf.estimator`.  Se você escreve seu código usando `tf.estimator` e tiver interesse em usar mais de uma máquina com alto desempenho, este tutorial é para você.

Antes de começar, pedimos que leia o guia [Estratégia de distribuição](../../guide/distributed_training.ipynb).  O [tutorial Treinamento com várias GPUs](./keras.ipynb) também é relevante, pois este tutorial usa o mesmo modelo.


## Configuração

Primeiro, configure o TensorFlow e as importações necessárias.

In [ ]:
import tensorflow_datasets as tfds
import tensorflow as tf

import os, json

Observação: a partir do TF 2.4, a estratégia espelhada multiworker não funciona com estimadores se a execução adiantada (eager) estiver ativada (que é o padrão). O erro no TF 2.4 é: `TypeError: cannot pickle '_thread.lock' object` (TypeError: não é possível conversar o objeto '_thread.lock'). Confira mais detalhes no [problema 46556](https://github.com/tensorflow/tensorflow/issues/46556). A solução alternativa é desativar a execução adiantada (eager)

In [ ]:
tf.compat.v1.disable_eager_execution()

## Função de entrada

Este tutorial usa o conjunto de dados MNIST dos [conjuntos de dados do TensorFlow](https://www.tensorflow.org/datasets).  Este código é similar ao [tutorial Treinamento com várias GPUs](./keras.ipynb), com uma diferença importante: ao usar o Estimador para treinamento multiworker, é necessário fragmentar o conjunto de dados pelo número de workers para garantir a convergência do modelo.  Os dados de entrada são fragmentados pelo índice dos workers para que cada worker processe <code>1/num_workers</code> porções distintas do conjunto de dados.

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

def input_fn(mode, input_context=None):
  datasets, info = tfds.load(name='mnist',
                                with_info=True,
                                as_supervised=True)
  mnist_dataset = (datasets['train'] if mode == tf.estimator.ModeKeys.TRAIN else
                   datasets['test'])

  def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

  if input_context:
    mnist_dataset = mnist_dataset.shard(input_context.num_input_pipelines,
                                        input_context.input_pipeline_id)
  return mnist_dataset.map(scale).cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

Outra estratégia interessante para conseguir convergência é misturar o dataset com sementes distintas em cada worker.

## Configuração multiworker

Uma das principais diferentes neste tutorial em comparação ao [tutorial Treinamento com várias GPUs](./keras.ipynb) é a configuração multiworker.  A variável de ambiente `'TF_CONFIG'` é a forma padrão de especificar a configuração de cluster em cada worker que faz parte do cluster.

Existem dois componentes de `TF_CONFIG`: `cluster` e `task` (tarefa). `cluster` fornece informações sobre todo o cluster, ou seja, os workers e os servidores de parâmetros do cluster. `task` fornece informações sobre a tarefa atual. O primeiro componente, `cluster`, é o mesmo para todos os workers e servidores de parâmetros do cluster, e o segundo componente, `task`, é diferente em cada worker e servidor de parâmetros, especificando seu próprio `type` (tipo) e `index` (índice). Neste exemplo, o `type` da tarefa é `worker` e o `index` da tarefa é `0`.

Para fins de ilustração, este tutorial mostra como definir uma variável `TF_CONFIG` com dois workers no `localhost`.  Na prática, você criaria diversos workers em um endereço IP e porta externos e definiria `TF_CONFIG` em cada worker da forma adequada, ou seja, modificado o `index` da tarefa.

Aviso: *não execute o código abaixo no Colab.*  O runtime do TensorFlow tentará criar um servidor gRPC no endereço IP e porta especificados, e provavelmente haverá uma falha. Confira a [versão deste tutorial para Keras](multi_worker_with_keras.ipynb) se quiser ver como fazer o teste de execução de vários workers em uma única máquina.

```
os.environ['TF_CONFIG'] = json.dumps({
    'cluster': {
        'worker': ["localhost:12345", "localhost:23456"]
    },
    'task': {'type': 'worker', 'index': 0}
})
```


## Definição do modelo

Escreva as camadas, o otimizador e a função de perda para o treinamento.  Este tutorial define o modelo com camadas do Keras, similar ao [tutorial Treinamento com várias GPUs](./keras.ipynb).

In [ ]:
LEARNING_RATE = 1e-4
def model_fn(features, labels, mode):
  model = tf.keras.Sequential([
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  logits = model(features, training=False)

  if mode == tf.estimator.ModeKeys.PREDICT:
    predictions = {'logits': logits}
    return tf.estimator.EstimatorSpec(labels=labels, predictions=predictions)

  optimizer = tf.compat.v1.train.GradientDescentOptimizer(
      learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction=tf.keras.losses.Reduction.NONE)(labels, logits)
  loss = tf.reduce_sum(loss) * (1. / BATCH_SIZE)
  if mode == tf.estimator.ModeKeys.EVAL:
    return tf.estimator.EstimatorSpec(mode, loss=loss)

  return tf.estimator.EstimatorSpec(
      mode=mode,
      loss=loss,
      train_op=optimizer.minimize(
          loss, tf.compat.v1.train.get_or_create_global_step()))

Observação: embora a taxa de aprendizado seja fixa neste exemplo, pode ser necessário ajustá-la com base no tamanho global de lote.

## MultiWorkerMirroredStrategy

Para treinar o modelo, use uma instância de `tf.distribute.experimental.MultiWorkerMirroredStrategy`.  O `MultiWorkerMirroredStrategy` cria cópias de todas as variáveis nas camadas do modelo em cada dispositivo, em todos os workers.  Ele usa `CollectiveOps`, uma operação do TensorFlow para comunicação coletiva, para agregar os gradientes e manter as variáveis sincronizadas.  Confira mais detalhes sobre essa estratégia no [guia `tf.distribute.Strategy`](../../guide/distributed_training.ipynb).

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

## Treinamento e avaliação do modelo

Agora, especifique a estratégia de distribuição no `RunConfig` para o estimador, depois chame `tf.estimator.train_and_evaluate` para treinar e avaliar o modelo.  Este tutorial distribui somente o treinamento especificando a estratégia com `train_distribute`.  Também é possível distribuir a avaliação com `eval_distribute`.

In [ ]:
config = tf.estimator.RunConfig(train_distribute=strategy)

classifier = tf.estimator.Estimator(
    model_fn=model_fn, model_dir='/tmp/multiworker', config=config)
tf.estimator.train_and_evaluate(
    classifier,
    train_spec=tf.estimator.TrainSpec(input_fn=input_fn),
    eval_spec=tf.estimator.EvalSpec(input_fn=input_fn)
)

## Otimização do desempenho do treinamento

Agora você tem um modelo e um Estimador multiworker com base no `tf.distribute.Strategy`.  Você pode tentar as seguintes técnicas para otimizar o desempenho do treinamento multiworker:

- *Aumente o tamanho do lote:* o tamanho do lote especificado aqui é por GPU.  De forma geral, recomenda-se o maior tamanho de lote que cabe na memória das GPUs.

- *Converta as variáveis:* converta as variáveis para `tf.float`, se possível.  O modelo oficial ResNet inclui [um exemplo](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) de como fazer a conversão.

- *Use comunicação coletiva:* o `MultiWorkerMirroredStrategy` tem diversas [implementações de comunicação coletiva](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py).

    - O `RING` usa coletivos com base em anel usando o gRPC como a camada de comunicação entre hosts.
    - O `NCCL` usa o [NCCL da Nvidia](https://developer.nvidia.com/nccl) para implementar coletivos.
    - `AUTO` adia a escolha para o runtime.

    A melhor escolha de implementação de coletivos depende do número e tipo de GPUs, além da interconexão de rede no cluster.  Para sobrescrever a escolha automática, especifique um valor válido para o parâmetro `communication` (comunicação) do construtor de `MultiWorkerMirroredStrategy`, por exemplo: `communication=tf.distribute.experimental.CollectiveCommunication.NCC`.

Confira a [seção Desempenho](../../guide/function.ipynb) do guia para saber mais sobre outras estratégias e [ferramentas](../../guide/profiler.md) que você pode usar para otimizar o desempenho dos seus modelos do TensorFlow.


## Outros exemplos de código

1. [Código completo](https://github.com/tensorflow/ecosystem/tree/master/distribution_strategy) para treinamento multiworker no ecossistema do TensorFlow usando modelos de Kubernetes. Esse exemplo começa com um modelo do Keras, que é convertido em um Estimador usando a API <code>tf.keras.estimator.model_to_estimator</code>.
2. [Modelos oficiais](https://github.com/tensorflow/models/tree/master/official), muitos dos quais podem ser configurados para executarem diversas estratégias de distribuição.
